### Benchmark: Theis storage test

In [1]:
import numpy as np
from scipy.integrate import quad, solve_ivp
from scipy.special import exp1

In [2]:
Q = -100
npor = 0.3
k = 10
H = 10
T = k * H
Ss = 1e-4
S = Ss * H

In [3]:
def headtheis(r, t, T=T, S=S, Q=Q):
    u = r**2 * S / (4 * T * t)
    h = -Q / (4 * np.pi * T) * exp1(u)
    return h


def voltheis(r, t, T, S, Q):
    u = r**2 * S / (4 * T * t)
    h = -Q / (4 * np.pi * T) * exp1(u)
    vol = h * 2 * np.pi * r
    return vol * S

In [4]:
# demonstrate headtheis works correctly
quad(voltheis, a=1e-3, b=10000, args=(10, T, S, Q))  # gives 1000 m^3

(999.999999994425, 1.4198836879586452e-06)

In [5]:
def volume(r, t, T=T, S=S, Q=Q):
    return quad(voltheis, a=1e-3, b=r, args=(10, T, S, Q))[0] + npor * np.pi * (r**2)

In [6]:
def vxytheis(t, xy):
    x, y = xy
    r = np.sqrt(x**2 + y**2)
    u = S * r**2 / (4 * T * t)
    Qr = -Q / (2 * np.pi) / r * np.exp(-u)
    vr = Qr / (H * npor)
    vx = vr * x / r
    vy = vr * y / r
    return np.array([vx, vy])

In [7]:
def vxytheisnew(t, xy):
    x, y = xy
    r = np.sqrt(x**2 + y**2)
    u = S * r**2 / (4 * T * t)
    Qr = -Q / (2 * np.pi) / r * np.exp(-u)
    vr = Qr / (H * (npor + S * headtheis(x, t)))
    vx = vr * x / r
    vy = vr * y / r
    return np.array([vx, vy])

In [8]:
# trace pathline for 10 days
path0 = solve_ivp(vxytheis, (1e-5, 10), y0=[1e-5, 0], method="DOP853")
R0 = path0.y[0, -1]
R0, np.pi * R0**2 * npor * H

(10.300504380350029, 999.9726219155026)

In [9]:
path0 = solve_ivp(vxytheisnew, (1e-5, 10), y0=[1e-5, 0], method="DOP853")
R1 = path0.y[0, -1]
R1, np.pi * R1**2 * npor * H

(10.286920903323537, 997.3369938953231)

In [10]:
headtheis(5, 10) * S

0.0009076383332409225

In [11]:
npor * H

3.0

In [12]:
R = path0.y[0, -1]
print("R, volume", R, np.pi * R**2 * npor * H)

R, volume 10.286920903323537 997.3369938953231


In [13]:
from scipy.integrate import quad

quad(headtheis, 1e-5, R, args=(100, T, S, Q))[0] + np.pi * R**2 * npor * H

1009.0147425061979

In [14]:
t0 = 0
t1 = 10


def vxytheis2(t, xy):
    x, y = xy
    r = np.sqrt(x**2 + y**2)
    u1 = S * r**2 / (4 * T * (t - t0))
    u2 = S * r**2 / (4 * T * (t - t1))
    Qr = -Q / (2 * np.pi) / r * np.exp(-u1)
    if t >= t1:
        Qr += 2 * Q / (2 * np.pi) / r * np.exp(-u2)
    vr = Qr / (H * npor)
    vx = vr * x / r
    vy = vr * y / r
    return np.array([vx, vy])

In [15]:
path1 = solve_ivp(vxytheis2, (10 + 1e-6, 20), y0=[R, 0], method="DOP853")
path1.y[0, -1]

0.23642465667948978

In [16]:
path1

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 194
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([10.000001  , 10.27008746, 10.59652189, 12.68413034, 17.0030137 ,
       18.78126275, 19.51325123, 19.81416749, 19.9371275 , 19.98633926,
       20.        ])
 t_events: None
        y: array([[10.2869209 , 10.16266029,  9.99100995,  8.81345225,  5.64399507,
         3.60375877,  2.28483344,  1.42395422,  0.85029311,  0.44815448,
         0.23642466],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])
 y_events: None

In [17]:
vxytheis2(12 + 1e-6, (10, 0))

array([-0.53039491, -0.        ])